In [2]:
import pandas as pd
import numpy as np
import random
import math
from random import shuffle 

class iNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.weights=None
        
class oNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.bias=None
        self.error=None
        
class innerNode:
    def __init__(self):
        self.input=0
        self.output=None
        self.bias=None
        self.weights=None
        self.error=None
        
class MLP:
    def __init__(self,inodes,n,innerlayers,bias,weight1,weight2,learning_rate):
        self.rate=learning_rate
        self.outputnode=oNode()
        self.layers=[[innerNode() for i in range(n)] for j in range(innerlayers)]
        self.layers=[[iNode() for i in range(inodes)]]+self.layers+[[self.outputnode]]
        for layer in range(len(self.layers)):
            if layer==0:
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].weights=[random.uniform(-1,1) for i in range(len(self.layers[layer+1]))]
            
            elif layer==len(self.layers)-1:
                for node in range(len(self.layers[layer])):
                    #self.layers[layer][node].weights=[weight2 for i in range(len(self.layers[layer+1]))]
                    self.layers[layer][node].bias=0        #change
                    
            else:
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].weights=[random.uniform(-1,1) for i in range(len(self.layers[layer+1]))]
                    self.layers[layer][node].bias=random.random()
                    
    def fun(self,x):
        return 1/(1+math.exp(-1*x))
    
    def derivative(self,x):
        return (x.output*(1-x.output))
    
    def classification(self,x):
        if x>0.5:
            return 1
        return 0
                    
    def train(self,ltrain,ztrain):
        number=0
        H=[]
        for input in ltrain:
            H.append([])
            for node in range(len(input)):
                self.layers[0][node].output=input[node]
            
            for layer in range(1,len(self.layers)-1):           #forward propagation
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].input=self.layers[layer][node].bias
                    for i in self.layers[layer-1]:
                        self.layers[layer][node].input+=(i.weights[node]*i.output)
                    self.layers[layer][node].output=self.fun(self.layers[layer][node].input)
                    H[-1].append(self.layers[layer][node].output)
            #self.outputnode.output=self.classification(self.outputnode.output)
            
            """self.outputnode.error=(ztrain[number]-self.outputnode.output)   #back propagation
            for layer in range(len(self.layers)-2,-1,-1):
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].error=0
                    for j in range(len(self.layers[layer+1])):
                        self.layers[layer][node].error+=(self.layers[layer][node].weights[j]*self.layers[layer+1][j].error)
                    self.layers[layer][node].error*=self.derivative(self.layers[layer][node])
                                                                    
            for i in range(0,len(self.layers)-1):
                for j in range(len(self.layers[i])):
                    for k in range(len(self.layers[i][j].weights)):
                        self.layers[i][j].weights[k]+=self.rate*self.layers[i+1][k].error*self.layers[i][j].output
                        
                    if i!=0:
                        self.layers[i][j].bias+=(self.rate*self.layers[i][j].error)"""
            number+=1
        return(H)
            
    def test(self,ltest,ztest):
        number=0
        count=0
        tp=0
        tn=0
        fp=0
        fn=0
        #print(ztest)
        for input in ltest:
            for node in range(len(input)):
                self.layers[0][node].output=input[node]
            
            for layer in range(1,len(self.layers)):           #forward propagation
                for node in range(len(self.layers[layer])):
                    self.layers[layer][node].input=self.layers[layer][node].bias
                    for i in self.layers[layer-1]:
                        self.layers[layer][node].input+=(i.weights[node]*i.output)
                    self.layers[layer][node].output=self.fun(self.layers[layer][node].input)
            self.outputnode.output=self.classification(self.outputnode.output)
            #print(self.outputnode.output)
            if self.outputnode.output==ztest[number]:
                count+=1
                if ztest[number]==1:
                    tp+=1
                else:
                    tn+=1
            elif ztest[number]==1:
                fn+=1
            else:
                fp+=1
            number+=1
        print('accuracy =',count/len(ztest)*100,'%')
        print('precision(+) =',tp/(tp+fp))
        print('recall(+) =',tp/(tp+fn))
        print('precision(-) =',tn/(tn+fn))
        print('recall(-) =',tn/(tn+fp))
        
    def findB(self,H,T):
        Htran=np.transpose(H)
        prod=np.dot(H,Htran)
        row=len(prod)
        col=len(prod[0])
        lam_invmat=np.zeros((row,col),float)
        np.fill_diagonal(lam_invmat,random.random())
        if np.linalg.det(prod)!=0:
            temp=np.sum([lam_invmat,prod],axis=0)
            #print(lam_invmat)
            #print(prod)
            temp=np.linalg.inv(temp)
            B=np.dot(Htran,temp)
            B=np.dot(B,T)
        else:
            prod=np.dot(Htran,H)
            row=len(prod)
            col=len(prod[0])
            lam_invmat=np.zeros((row,col),float)
            np.fill_diagonal(lam_invmat,random.random())
            temp=np.sum([lam_invmat,prod],axis=0)
            #print(lam_invmat)
            #print(prod)
            temp=np.linalg.inv(temp)
            B=np.dot(temp,Htran)
            B=np.dot(B,T)
        #print('len=',len(self.layers[-2]))
        #for i in range(len(self.layers[-2])):
            #self.layers[-2][i].weights=[B[i]]
        return B
    
    def sparse_encoder(self,A,b,lam,itrs):
        prod= np.dot(np.transpose(A),A)
        eigvals,eigvecs = np.linalg.eig(prod)
        Li=1/max(eigvals)
        alpha = lam * Li
        m=len(A[1])
        n=len(b[1])
        x=np.zeros((m,n),float)
        yk=x[:][:]
        tk=1
        L1 = 2 * Li * np.array(prod)
        L2 = np.dot(2 * Li * np.array(np.transpose(A)) , np.array(b))
        for i in range(itrs):
            ck= np.array(yk)- np.dot(L1,yk) + np.array(L2)
            x1= np.multiply(np.absolute(ck)-alpha,np.sign(ck))
            tk1= 0.5 + 0.5 * ( (1 + 4 * (tk**2)) ** 0.5 )
            tt = (tk-1)/tk1
            yk = x1 + tt*np.array((np.array(x) - np.array(x1)))
            tk = tk1
            x = x1
        return x
    
    def train2(self,ltrain,ztrain):
        b1=[[random.random() for i in range(5)] for j in range(len(ltrain[0]))]
        A1=np.dot(ltrain,b1)
        beta1= np.transpose(self.sparse_encoder(A1,ltrain,0.001,50))
        
        T1=np.dot(ltrain,beta1)
        
        H2=T1
        b2=[[random.random() for i in range(10)] for j in range(5)]
        A2= np.dot(H2,b2)
        beta2= np.transpose(self.sparse_encoder(A2,H2,0.001,50))
        
        T2=np.dot(H2,beta2)
        H3=T2
        B=self.findB(H3,ztrain)
        return [beta1,beta2,B]
        
    def test2(self,ltest,ztest,beta1,beta2,B):
        O1=np.dot(ltest,beta1)
        O2=np.dot(O1,beta2)
        O3=np.dot(O2,B)
        #print(O3)
        count=0
        for i in range(len(ztest)):
            if ztest[i]==self.classification(O3[i]):
                count+=1
        print('accuracy=',count*100/len(ztest))
        
            
        


data=pd.read_csv('C:\\Users\\Rakesh\\SC\\lab2\\SPECT.csv')
copy=data.values.tolist()
shuffle(copy)
#copy=data[:][:]
ltrain=[]
ztrain=[]
ltest=[]
ztest=[]
dict={}
c=0
n=data.iloc[:,1].count()

for i in range(n):            # dividing train and test dataset
    row=random.randint(0,len(copy)-1)
    if i<0.8*n:
        ltrain.append(copy[row][1:])
        ztrain.append(copy[row][0])
        if copy[row][0] not in dict:
            dict[copy[row][0]]=c
            c+=1
        copy.remove(copy[row])
    else:
        ltest.append(copy[row][1:])
        ztest.append(copy[row][0])
        copy.remove(copy[row])
        
for i in range(len(ztrain)): #changing categorical variables to numbers
    ztrain[i]=dict[ztrain[i]]
for i in range(len(ztest)):
    ztest[i]=dict[ztest[i]]  
inputs=len(ltrain[0])
M=MLP(inputs,200,1,1/6,1/(5*inputs),1/5,0.2)
[beta1,beta2,B]=M.train2(ltrain,ztrain)
#B=M.findB(H,ztrain)
M.test2(ltest,ztest,beta1,beta2,B)
#print(M.layers[-2][0].weights)
#print(B)

accuracy= 86.79245283018868


In [6]:
import numpy as np
#centered_matrix = X - X.mean(axis=1)[:, np.newaxis]
c=[[1,2],[3,4]]
ctran=np.transpose(c)
cov = np.dot(c,ctran)
eigvals,e= np.linalg.eig(cov)

In [7]:
eigvals

array([ 0.13393125, 29.86606875])

In [3]:
max(eigvals)

29.866068747318504

In [8]:
2*np.arange(5)

array([0, 2, 4, 6, 8])

In [13]:
A=[[1,2,3],[2,3,4]]
2*np.array(A)#.reshape(2,3)

array([[2, 4, 6],
       [4, 6, 8]])

In [15]:
B=[[1,2,3],[2,3,4]]
np.array(A) + np.array(B)

array([[2, 4, 6],
       [4, 6, 8]])

In [16]:
A=np.array(A)

In [17]:
A

array([[1, 2, 3],
       [2, 3, 4]])

In [18]:
np.array(A)

array([[1, 2, 3],
       [2, 3, 4]])

In [23]:
B=[[1,2,2],[3,3,4]]


In [24]:
np.multiply(A,B)

array([[ 1,  4,  6],
       [ 6,  9, 16]])

In [25]:
abs(-1)

1

In [27]:
max([1,2],0)

TypeError: '>' not supported between instances of 'int' and 'list'

In [28]:
A=[[1,2,3],[1,5,6]]
AA = (np.transpose(A)) * A

Lf = max(eig(AA))
Li = 1 / Lf
alp = lam * Li
m = size(A, 2)
n = size(b, 2)
x = zeros(m, n)
yk = x
tk = 1
L1 = 2 * Li * AA
L2 = 2 * Li * A.cT * b
# tic
for i in mslice[1:itrs]:
    ck = yk - L1 * yk + L2
    x1 = (max(abs(ck) - alp, 0)) *elmul* sign(ck)
    tk1 = 0.5 + 0.5 * sqrt(1 + 4 * tk ** 2)
    tt = (tk - 1) / tk1
    yk = x1 + tt * (x - x1)
    tk = tk1
    x = x1
# toc

AttributeError: 'list' object has no attribute 'cT'

In [30]:
2*np.array([[1,2],[3,4]])

array([[2, 4],
       [6, 8]])

In [8]:
a=[[1,-2],[3,-4]]
np.absolute(a)

array([[1, 2],
       [3, 4]])